# Module 1 Homework

In [2]:
import math
import pickle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error 

In [3]:
df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [4]:
df_len = len(df)
df_len

3066766

In [5]:
print(f'January data has {len(df.columns)} columns.')

January data has 19 columns.


In [6]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime)

# Convert to minutes
df.duration = df.duration.apply(lambda x: x.total_seconds()/60.0)
df.duration.head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration, dtype: float64

In [7]:
# Get std of duration
df.duration.describe()['std']

np.float64(42.59435124195458)

In [8]:
# Remove outliers
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [9]:
df_len_after = len(df)
df_len_after

3009173

In [10]:
# Percent of records retained after removing outliers
df_len_after/df_len*100

98.1220282212598

In [11]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [12]:
categorical = ['PULocationID', 'DOLocationID']

df.loc[:, categorical] = df.loc[:, categorical].astype(str)

df.dtypes

/tmp/ipykernel_6597/2230725837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:, categorical].astype(str)
/tmp/ipykernel_6597/2230725837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:, categorical].astype(str)


VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [13]:
dv = DictVectorizer()

train_dicts= df[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

In [14]:
print(f'X_train has {X_train.shape[1]} columns.')

X_train has 515 columns.


In [15]:
# Target 
y_train = df.duration.values

In [16]:
linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = linear_model.predict(X_train)
print(f'Train RMSE: {math.sqrt(mean_squared_error(y_train, y_pred))}')

Train RMSE: 7.649261932106969


In [18]:
# Function that does everythin above: Can be used for validation dataset

def process_dataset(path):
    df = pd.read_parquet(path)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime)
    
    # Convert datetime to minutes
    df.duration = df.duration.apply(lambda x: x.total_seconds()/60.0)

    categorical = ['PULocationID', 'DOLocationID']

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df.loc[:, categorical] = df.loc[:, categorical].astype(str)

    return df

In [19]:
df_val = process_dataset('data/yellow_tripdata_2023-02.parquet')

/tmp/ipykernel_6597/344273269.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:, categorical].astype(str)
/tmp/ipykernel_6597/344273269.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '26' '145' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:, categorical].astype(str)


In [20]:
len(df_val)

2855951

In [21]:
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)
y_val = df_val.duration.values

In [22]:
y_pred = linear_model.predict(X_val)
print(f'Val RMSE: {math.sqrt(mean_squared_error(y_val, y_pred))}')

Val RMSE: 7.811818743246608
